In [5]:
import pyspark
from pyspark.context import SparkContext
from pyspark.sql import SparkSession, SQLContext

spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = spark.sparkContext

In [6]:
df = spark.read.load("../data/rhci-moby.parquet")

In [7]:
df.count()

223014384

In [8]:
from pyspark.sql.functions import *
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, StopWordsRemover

In [10]:
df.printSchema()

root
 |-- @timestamp: timestamp (nullable = true)
 |-- __label: string (nullable = true)
 |-- ci_agent: struct (nullable = true)
 |    |-- label: string (nullable = true)
 |    |-- name: string (nullable = true)
 |-- ci_job: struct (nullable = true)
 |    |-- build_id: integer (nullable = true)
 |    |-- full_url: string (nullable = true)
 |    |-- log_url: string (nullable = true)
 |    |-- name: string (nullable = true)
 |    |-- phase: string (nullable = true)
 |    |-- status: string (nullable = true)
 |-- ci_master: struct (nullable = true)
 |    |-- hostname: string (nullable = true)
 |-- file: string (nullable = true)
 |-- geoip: struct (nullable = true)
 |    |-- location: struct (nullable = true)
 |    |    |-- lat: double (nullable = true)
 |    |    |-- lon: double (nullable = true)
 |-- hostname: string (nullable = true)
 |-- ipaddr4: string (nullable = true)
 |-- ipaddr6: string (nullable = true)
 |-- level: string (nullable = true)
 |-- message: string (nullable = true)
 

In [11]:
pro = df.filter(df["message"].isNotNull()).select(df["ci_job.build_id"], when(df["ci_job.status"] == "SUCCESS", 0).otherwise(1).alias("status"), lower(df["message"]).alias("message"))

In [12]:
tokenizer = Tokenizer().setInputCol("message").setOutputCol("rawTokens")
remover = StopWordsRemover().setInputCol("rawTokens").setOutputCol("tokens")

In [13]:
exploded = remover.transform(tokenizer.transform(pro)).select("build_id", "status", explode(col("tokens")).alias("token"))

In [14]:
imploded = exploded.distinct().filter(col("token").isNotNull().alias("token")).groupBy("build_id", "status") \
    .agg(col("build_id"), col("status"), collect_set(col("token")))

In [16]:
pro.repartition(1024).write.mode("overwrite").save("../data/rhci-pro.parquet")

Py4JJavaError: An error occurred while calling o155.save.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$$anonfun$write$1.apply$mcV$sp(FileFormatWriter.scala:147)
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$$anonfun$write$1.apply(FileFormatWriter.scala:121)
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$$anonfun$write$1.apply(FileFormatWriter.scala:121)
	at org.apache.spark.sql.execution.SQLExecution$.withNewExecutionId(SQLExecution.scala:57)
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:121)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:101)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.sideEffectResult$lzycompute(commands.scala:58)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.sideEffectResult(commands.scala:56)
	at org.apach

In [68]:
imploded.take(5)


[Row(build_id=196, status=0, build_id=196, status=0, collect_set(token)=['libiscsi-1.9.0-7.el7.x86_64', '18:16:59', '10.8.173.82', '02:16:21,258', '87.2m', '13:22:20,016', 'libnfsidmap-0.25-15.el7.x86_64', './testconfigdbsnapshots/2016-08-16_14-36-34/qci_1.0_rhev_conn_api-mlibvirt-qci_ose_cfme-smoke-61dffb37-df61-40ea-99be-72345ffd45fa.yaml', '--disk="path=/tmp/shqxgghwfc_test_runner.qcow2,device=disk,bus=virtio,format=qcow2"', '13:16:26,608', 'reboot]', '06:43:41,314', '02:27:25,641', '\'qci_1.0_rhev_conn_api-mlibvirt-qci-smoke-1371a-stable-provision\'}",', 'editing', '\\"memory>=60000\\"]",', "'fe80:52:0:800::1ffe'},", '18:09:18,174', '15:44:45,328', '13:45:28,281', '22:03:36,794', 'class', '20:19:33,950', 'pitteardownurl=https://rhcs-jenkins.rhev-ci-vms.eng.rdu2.redhat.com/job/qci_1.0_rhev_conn_gui-mlibvirt-qci_ose_cfme-smoke-56448-stable-teardown/', '0x33dca28>,', '02:27:52,865', '09:51:06,506', '08:20:28,607', 'atomic-openshift-master-api', '311/512', '10:40:33', '13:33:17,', "'rc